In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
import tensorflow_datasets as tfds

In [4]:
iris, iris_info = tfds.load('iris', with_info=True)

In [5]:
iris

{'train': <PrefetchDataset shapes: {features: (4,), label: ()}, types: {features: tf.float32, label: tf.int64}>}

In [6]:
tf.random.set_seed(1)
ds_orig = iris['train']

In [7]:
# the dataset is not split into train and test
# partition then shuffle only once to prevent mixing of train and test sets
ds_orig = ds_orig.shuffle(150, reshuffle_each_iteration = False)
ds_train_orig = ds_orig.take(100)
ds_test = ds_orig.skip(100)

In [8]:
# convert the dictionary into a tuple using map()
ds_train_orig = ds_train_orig.map(lambda x:(x['features'],x['label']))
ds_test = ds_test.map(lambda x:(x['features'], x['label']))

In [9]:
iris_model = tf.keras.Sequential([tf.keras.layers.Dense(
    16,activation='sigmoid', name='fc1',input_shape=(4,)),
                                tf.keras.layers.Dense(3,name='fc2',
                                                     activation='softmax')])

In [10]:
iris_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
fc1 (Dense)                  (None, 16)                80        
_________________________________________________________________
fc2 (Dense)                  (None, 3)                 51        
Total params: 131
Trainable params: 131
Non-trainable params: 0
_________________________________________________________________


In [11]:
iris_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

In [12]:
num_epochs = 100
training_size = 100
batch_size = 2
steps_per_epoch = np.ceil(training_size / batch_size)
ds_train = ds_train_orig.shuffle(buffer_size=training_size)
ds_train = ds_train.repeat()
ds_train = ds_train.batch(batch_size = batch_size)
ds_train = ds_train.prefetch(buffer_size=1000)
history = iris_model.fit(ds_train, epochs=num_epochs,
                         steps_per_epoch=steps_per_epoch, verbose=0)

InternalError:  Blas GEMM launch failed : a.shape=(2, 4), b.shape=(4, 16), m=2, n=16, k=4
	 [[node sequential/fc1/MatMul (defined at <ipython-input-12-21c1f349d442>:9) ]] [Op:__inference_train_function_639]

Function call stack:
train_function
